<a href="https://colab.research.google.com/github/SanjayKumar16225/2022-ML/blob/main/2nd_Jan_Sales_Forecasting_Time_Series_Forecasting_by_Prophet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
train_df=pd.read_csv('/content/drive/MyDrive/ML2022/Jan-2022/train.csv',parse_dates=["Date"])
test_df=pd.read_csv('/content/drive/MyDrive/ML2022/Jan-2022/test.csv',parse_dates=["Date"])

In [ ]:
train_df.head()

In [ ]:
train_df_cln=train_df.copy()
train_df_cln=train_df.groupby(["Date"]).agg({"Weekly_Sales":["sum"]}).reset_index()
train_df_cln.columns=['ds','y']


In [ ]:
"""
1. aggregate to weekly data
2. rename columns for Prophet Model
"""

test_df_cln=test_df.copy()
test_df_cln=test_df["Date"].drop_duplicates().sort_values().to_frame()
test_df_cln.columns=["ds"]
test_df_cln

In [ ]:
train_df_cln

In [ ]:
plt.figure(figsize=(20,5)) # chart size
plt.plot(train_df_cln["ds"], train_df_cln["y"])

In [ ]:
"""
1. decompose time series data: level, trend, seasonality
"""

import statsmodels.api as sm

sm.tsa.seasonal_decompose(train_df.groupby("Date")["Weekly_Sales"].sum(),  model="additive").plot().show()
sm.tsa.seasonal_decompose(train_df.groupby("Date")["Weekly_Sales"].sum(),  model="multiplicative").plot().show()
#sm.tsa.seasonal_decompose(train_df_cln["y"],period=52, model="additive").plot().show()
#sm.tsa.seasonal_decompose(train_df_cln["y"],period=52, model="multiplicative").plot().show()

In [ ]:
pip install Prophet

In [ ]:
"""
Prophet
"""

from prophet import Prophet

Prophet_Model=Prophet(yearly_seasonality=True, seasonality_mode="additive")
Prophet_Model.fit(train_df_cln)

In [ ]:
Y_P_Prophet_Model=Prophet_Model.predict(test_df_cln)
Y_P_Prophet_Model

In [ ]:
"""
1. evaluate forecast accuracy
2. prepare dataframe for performance_metrics function (same structure as cross_validation output)
"""

Y_Prophet_Model=Prophet_Model.predict(train_df_cln)
Y_Prophet_Model=Y_Prophet_Model[["ds","yhat","yhat_lower","yhat_upper"]]

Y_Prophet_Model=train_df_cln.merge(Y_Prophet_Model, how="left", left_on="ds", right_on="ds")
Y_Prophet_Model["cutoff"]=Y_Prophet_Model["ds"]
Y_Prophet_Model

In [ ]:
from prophet.diagnostics import performance_metrics
performance_metrics(Y_Prophet_Model)

In [ ]:
plt.figure(figsize=(20,5)) # chart size
plt.plot(Y_Prophet_Model["ds"], Y_Prophet_Model["y"], label="y")
plt.plot(Y_Prophet_Model["ds"], Y_Prophet_Model["yhat"], label="yhat")
plt.legend(loc='best')

In [ ]:
Prophet_Model.plot(Y_P_Prophet_Model, figsize=(10,3))
Prophet_Model.plot_components(Y_P_Prophet_Model, yearly_start=0, figsize=(10,5))

In [ ]:
from fbprophet.plot import plot_plotly, plot_components_plotly

plot_plotly(Prophet_Model, Y_P_Prophet_Model)

In [ ]:
plot_components_plotly(Prophet_Model, Y_P_Prophet_Model)

In [ ]:
### https://facebook.github.io/prophet/docs/quick_start.html#python-api